## assignment 1

In [1]:
import xml.etree.ElementTree as ET
from stemming.porter2 import stem
import re

### load stop word

In [2]:
stopword_path = 'englishST.txt'
def load_stopword(stopword_path):
    stopword_list = []
    with open(stopword_path, 'r',encoding='utf-8') as f1:
        stopword_list = [str(current_word).strip() for current_word in f1]
    return stopword_list

stopword_list = load_stopword(stopword_path)

In [3]:
def tokenisation_text(text):
    
    # simply remove every non-letter character
    #     del_punc = r'[^A-Za-z0-9_-]' # keep _ -
    #     del_punc = r'[^A-Za-z0-9_-\']' #keep _ - '
    del_punc = r'[\W]' # keep _

    text_nopunc = re.sub(del_punc,' ',text)
    tokenisation_list = text_nopunc.split()
    return tokenisation_list

def lower_word(word_list):
    return [current_word.strip().lower() for current_word in word_list]

def token_lower_nostop_stem_list(all_text, stopword_list):
    token_list = tokenisation_text(all_text)
    token_lowerlist = lower_word(token_list)
    token_lowerlist_nostop = [str(current_word) for current_word in token_lowerlist if str(current_word) not in stopword_list]
    stem_list = [stem(current_word) for current_word in token_lowerlist_nostop]

    return stem_list

In [4]:
def to_standard_xml(xml_file_path):
    new_file_name = 'standard_' + xml_file_path.split('/')[-1]
    new_file_path = data_folder + new_file_name

    with open(new_file_path,'w',encoding='utf-8') as new_file:
        with open(xml_file_path,'r') as original_file:
            new_file.write('<?xml version="1.0"?>\n')
            new_file.write('<sample>\n')
            for line in original_file:
                if(len(line.strip()) == 0):
                    continue
                new_file.write(line)
            new_file.write('</sample>')
    return new_file_path

def xml_sample(xml_file_path):
    tree = ET.parse(trec_samle_path)
    root = tree.getroot()
    return root

def merge_text(text1,text2):
    merged_text = text1 + text2
    return merged_text

def xml_all_text(xml_file_path,stopword_list,is_sample):
    if (is_sample):
        # sample
        standard_xml_path = to_standard_xml(sample_xml_path)
        tree = ET.parse(standard_xml_path)
        root = tree.getroot()
        all_text_list = [(child.find('DOCNO').text,token_lower_nostop_stem_list(child.find('Text').text, stopword_list)) for child in
                         root.findall("./DOC")]
    else:
        # trec
        standard_xml_path = to_standard_xml(trec_samle_path)
        tree = ET.parse(standard_xml_path)
        root = tree.getroot()
        all_text_list = [(child.find('DOCNO').text, token_lower_nostop_stem_list(merge_text(child.find('HEADLINE').text,child.find('TEXT').text), stopword_list)) for child in root.findall("./DOC")]

    return all_text_list

In [5]:
is_sample = 0
data_folder = 'collections/'
sample_txt_path = 'collections/sample.txt'
trec_samle_path = 'collections/trec.sample.xml'
sample_xml_path = 'collections/sample.xml'
stopword_filepath = 'englishST.txt'

all_text_list= [doc1_stemlist, doc2_stemlist, .......]

In [6]:
if(is_sample):
    all_text_list = xml_all_text(sample_xml_path,stopword_list,is_sample)
else:
    all_text_list = xml_all_text(trec_samle_path,stopword_list,is_sample)

generate inverted index

In [7]:
def inverted_index(all_text_list):
    '''
    return: # index dic[stem]={'doc_id':position}
    '''

    #get stem
    stem_set = set()
    for current_doc_tupple in all_text_list:
        for current_stem in current_doc_tupple[1]:
            stem_set.add(current_stem)

    index_dic = {}
    for current_stem in stem_set:
        current_stem_doc_position_dic = {}
        for current_doc_tupple in all_text_list:
            current_doc_id = current_doc_tupple[0]
            current_doc_text_list = current_doc_tupple[1]

            if (current_stem not in current_doc_text_list):
                continue

            position_list = [index+1 for index in range(len(current_doc_text_list))
                             if current_doc_text_list[index] == current_stem]
            current_stem_doc_position_dic[str(current_doc_id)] = position_list
        index_dic[current_stem] = current_stem_doc_position_dic
    return index_dic

In [8]:
index_dic = inverted_index(all_text_list)

In [9]:
def output_index(index_dic,file_path):
    sorted_index_dic_list = sorted(index_dic.items(),key=lambda x:x[0],reverse=False)
    with open(file_path,'w',encoding='utf-8') as file_1:
        for current_tupple in sorted_index_dic_list:
            current_stem = current_tupple[0]
            current_stem_doc_position_dic = current_tupple[1]
            file_1.write(current_stem)
            file_1.write(':\n')
            for current_doc_id,current_doc_id_position_list in current_stem_doc_position_dic.items():
                file_1.write('\t')
                file_1.write(str(current_doc_id))
                file_1.write(': ')
                current_doc_id_position_list_new = map(lambda x: str(x), current_doc_id_position_list)
                # print(','.join(current_doc_id_position_list_new))
                file_1.write(','.join(current_doc_id_position_list_new))
                file_1.write('\n')
            file_1.write('\n')

    return

output inverted dic

In [10]:
output_index_path = 'index.txt'
output_index(index_dic,output_index_path)

load inverted index from a file

In [11]:
def load_inverted_index(index_path):
    result_inverted_index = {}
    with open(index_path,'r') as f1:
        for line in f1:
            line = line.strip()
            if(line.endswith(':')):
                current_stem = line.replace(':','')
                result_inverted_index[current_stem] = {}
                continue
                
            if(len(line)==0):
                continue
                
            temp_split_list = line.split(': ')
            current_doc_id,str_position_list = temp_split_list[0],temp_split_list[1]
            current_position_list = [int(current_position) for current_position in str_position_list.split(',')]
            result_inverted_index[current_stem][current_doc_id] = current_position_list
    return result_inverted_index

In [12]:
inverted_index_path = 'index.txt'
loaded_inverted_index = load_inverted_index(inverted_index_path)

In [14]:
def get_doc_id_set(current_inverted_dic):
    doc_id_set = set()
    for current_stem,current_doc_position_dic in current_inverted_dic.items():
        for current_doc_id in current_doc_position_dic.keys():
            doc_id_set.add(str(current_doc_id))
    return doc_id_set

def query_word(current_inverted_dic,current_word,is_not=0):
    current_word_stem = stem(current_word.lower())
    if(is_not):
        for current_index_stem,current_index_stem_position in current_inverted_dic.items():
            if(current_word_stem == current_index_stem):
                doc_id_set = get_doc_id_set(current_inverted_dic)
                stem_doc_list = set(current_index_stem_position.keys())
                return list(doc_id_set.difference(stem_doc_list))
    else:
        for current_index_stem,current_index_stem_position in current_inverted_dic.items():
            if(current_word_stem == current_index_stem):
                return list(current_index_stem_position.keys())
    raise RuntimeError('Query not found.')
    return

# OR
def union_list(a,b):
    result_list = list(set(a).union(set(b)))
    result_list.sort(key=lambda i:int(i))
    return result_list

# AND
def intersection_list(a,b):
    result_list = list(set(a).intersection(set(b)))
    result_list.sort(key=lambda i:int(i))
    return result_list

def probability_query(doc_word_pos1,doc_word_pos2,current_distance):
    result_list = []
    for current_docid_1,current_positionlist_1 in doc_word_pos1.items():
        if(current_docid_1 not in doc_word_pos2.keys()):
            continue
        current_positionlist_2 = doc_word_pos2[current_docid_1]
        i = 0
        j = 0

        while((i <= len(current_positionlist_1) - 1) and (j <= len(current_positionlist_2) - 1)):
            if(int(current_positionlist_1[i]) > int(current_positionlist_2[j]) + current_distance):
                j+=1
                continue
            elif(int(current_positionlist_1[i]) < int(current_positionlist_2[j]) - current_distance):
                i+=1
                continue
            else:
                result_list.append(current_docid_1)
                break
    return result_list

#phrase
def phrase_query(current_inverted_dic,query_phrase):
    phrase_list = [ stem(current_word.strip()) for current_word in query_phrase.replace('"','').split()]
    result_list = probability_query(current_inverted_dic[phrase_list[0]],current_inverted_dic[phrase_list[1]],1)
    return result_list

# word position query
def word_position_query(current_inverted_dic, current_word):
    '''
    return: {'doc_id':[position_list]}
    '''
    if(current_word in current_inverted_dic.keys()):
        return current_inverted_dic[current_word]
    
    raise RuntimeError('Query not found.')
    return

def boolean_query(current_inverted_dic, current_query_word):
    is_NOT = 0
    
    #whether contains NOT
    if(current_query_word.startswith('NOT')):
        is_NOT = 1
        current_query_word = current_query_word.replace('NOT'.'').strip()
        
    if(current_query_word.startswith('"') and current_query_word.endswith('"')):
        result_list = phrase_query(loaded_index_dic,current_query_word)
    elif(current_query_word.startswith('#')):
        current_query_word_list = current_query_word.split('(')
        current_word_distance = int(current_query_word_list[0].replace('#','').strip())
        current_query_stem_list = [stem(temp_word.strip()) for temp_word in current_query_word_list[1].replace(')','').split(',')]
        result_list = probability_query(loaded_index_dic[current_query_stem_list[0]],loaded_index_dic[current_query_stem_list[1]],current_word_distance)
    else:
        result = query_word(loaded_index_dic,current_query_word)
    if(is_NOT):
        doc_id_set = get_doc_id_set(current_inverted_dic)
        result_list = list(doc_id_set.difference(set(result_list)))
    
    return result_list



In [ ]:
query_file_path = 'queries.boolean.txt'
operator_list = ['AND','OR']
query_result_list = []
if(query_file_path.split(.)[1] == 'boolean'):
    with open(query_file_path,'r') as query_file:
        for current_query_line in query_file:
            current_query_line = current_query_line.strip()
                if(len(current_querry_line) == 0):
                    continue
                current_query_id = current_query_line.split()[0] # get query id
                current_query = ' '.join(current_query_line.split()[1:]) # get query text
                
                #check whether contains any operator: 'AND' or 'OR'
                for current_operator in operator_list:
                    result_list = []
                    if(current_operator in current_query):
                        current_query_split_list = current_query.split(current_operator)
                        
                        for current_query_word in current_operator:
                            result_list.append(boolean_query(load_inverted_index,current_query_word))
                        
                        if(current_operator == 'AND'):
                            
                            
                        
                        break
                
        query_word_list =
elif(query_file_path.split(.)[1] == 'ranked'):
    
    current_result = tf_idf_weight(loaded_index_dic,current_query,is_stop,current_stopword_path)